In [7]:
from pathlib import Path
import json
import re
from tqdm import tqdm
from collections import defaultdict

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['LM(한국어)', '제목 짓기']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess
def replace_pattern_for_script(text):
    pattern = r'\((.*?)\)\/\((.*?)\) '
    replaced_text = re.sub(pattern, r'\2', text)
    return replaced_text

def replace_pattern_for_text(text):
    pattern = r'\((.*?)\)\/\((.*?)\) '
    replaced_text = re.sub(pattern, r'\1', text)
    return replaced_text
#### prepare for task preprocess end

#### task preprocess
for split in splits:
    text_dict = defaultdict(dict)
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            doc_id = line['id']
            title = line['title']
            text_id = line['index']
            domain = line['press_field']
            text = line['text']
            press_date = line['press_date']
            if 'info' not in text_dict[doc_id]:
                text_dict[doc_id]['info'] = {
                    'title': title,
                    'domain': domain,
                    'press_date': press_date
                }
                text_dict[doc_id]['text'] = {}
            text_dict[doc_id]['text'][text_id] = text
            #### data preprocess end 
    
    for key, value in text_dict.items():
        text = '\n'.join([v for k,v in sorted(value['text'].items())])
        doc_text = replace_pattern_for_text(text)
        doc_script = replace_pattern_for_script(text)
        title = value['info']['title']
        domain = value['info']['domain']
        press_date = value['info']['press_date']
        
        #### 제목 짓기
        data = {'input': {}, 'output': None}
        ## preprocess data from line
        data['input']['domain'] = domain
        data['input']['press_date'] = press_date
        data['input']['doc_text'] = doc_text
        data['output'] = {'title': title}
        ## preprocess data from line end
        task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
        # print(json.dumps(data, indent=4, ensure_ascii=False))
        #### 제목 짓기 end
        
        if doc_text == doc_script:
            continue
        #### LM(한국어)
        ## preprocess data from line
        data = {'text': None}
        doc = {
            '정보': value['info'],
            '대본': doc_text,
            '전사': doc_script,
        }
        data['text'] = json.dumps(doc, indent=4, ensure_ascii=False)
        ## preprocess data from line end
        task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
        # print(json.dumps(data, indent=4, ensure_ascii=False))
        #### LM(한국어) end
            
            
        
        
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 1/1 [00:00<00:00,  6.30it/s]
